### Importy

In [ ]:
# 📦 Importy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cmdstanpy
from cmdstanpy import CmdStanModel
import arviz as az
import scipy.stats as stats
# warnings.filterwarnings('ignore')

### Wczytanie danych i wybór potrzebnych wartości

In [ ]:
df_weather = pd.read_csv("data/weather.csv")
df_weather = df_weather[['timestamp', 'site_id', 'airTemperature' ]]
df_weather['timestamp'] = pd.to_datetime(df_weather['timestamp'])
df_weather.describe()
df_weather

In [ ]:
df_meta = pd.read_csv("data/metadata.csv")

# Filtruj tylko wiersze, gdzie electricity == "Yes" i wybierz odpowiednie kolumny
df_meta = df_meta[['site_id', 'building_id', 'sqm', 'lat', 'lng', 'timezone', 'electricity', 'industry', 'yearbuilt', 'numberoffloors', 'occupants']][df_meta['electricity'] == "Yes"]
df_meta = df_meta.drop(columns=['electricity'])

# Grupowanie z odpowiednimi funkcjami agregującymi
df_grouped = df_meta.groupby(["site_id", "building_id"]).agg({
    'sqm': 'sum',  # Sumuj powierzchnię
    'lat': 'first',  # Weź pierwszą wartość (bez sumowania)
    'lng': 'first',  # Weź pierwszą wartość
    'timezone': 'first',  # Zachowaj tekst
    'industry': 'first',  # Zachowaj tekst
    'yearbuilt': 'first',  # Nie sumuj roku budowy
    'numberoffloors': 'first',  # Nie sumuj liczby pięter
    'occupants': 'first',  # Nie sumuj liczby osób
}).reset_index()  # Reset index, jeśli chcesz mieć site_id i building_id jako kolumny

df_grouped

In [ ]:
df_electricity = pd.read_csv("data/electricity_cleaned.csv")
df_electricity.head()

In [ ]:
df_electricity_long = df_electricity.melt(
    id_vars=['timestamp'],  # Kolumna z czasem
    var_name='building_id',  # Nazwy kolumn staną się wartościami w tej kolumnie
    value_name='energy_consumption'  # Wartości z kolumn trafią tutaj
)

# Połączenie z metadanymi
df_merged = pd.merge(
    df_electricity_long,
    df_meta,
    on='building_id',
    how='outer'  # Zachowaj wszystkie wiersze z df_electricity_long
)

# Usunięcie wierszy z brakującymi wartościami energy_consumption
df_merged = df_merged.dropna(subset=['energy_consumption'])

# Reset indeksu (opcjonalne)
df_merged = df_merged.reset_index(drop=True)

# Wynik
df_merged.head()

In [ ]:
df_merged.head()

In [ ]:
df_merged['timestamp'] = pd.to_datetime(df_merged['timestamp'])
df_weather['timestamp'] = pd.to_datetime(df_weather['timestamp'])

# Połączenie danych
df_final = pd.merge(
    df_merged,
    df_weather,
    on=['timestamp', 'site_id'],
    how='left',
)

In [ ]:
# 2. Wybór 12 losowych budynków
unique_buildings = df_final['building_id'].unique()
random_buildings = np.random.choice(unique_buildings, size=12, replace=False)

# 3. Przygotowanie wykresów
fig, axes = plt.subplots(4, 3, figsize=(18, 15))
fig.suptitle('Histogramy zużycia energii z uwzględnieniem temperatury', fontsize=16, y=1.02)

for i, building_id in enumerate(random_buildings):
    # Dane dla wybranego budynku
    building_data = df_final[df_final['building_id'] == building_id]
    
    # Pozycja subplotu
    row, col = i // 3, i % 3
    ax = axes[row, col]
    
    # Histogram zużycia energii
    ax.hist(building_data['energy_consumption'], bins=30, color='blue', alpha=0.7)

    # Formatowanie
    ax.set_title(f'Budynek: {building_id}\nŚr. temp: {building_data["airTemperature"].mean():.1f}°C', fontsize=10)
    ax.set_xlabel('Zużycie energii', fontsize=8)
    ax.set_ylabel('Liczba wystąpień', fontsize=8, color='blue')
    ax.grid(True, linestyle='--', alpha=0.5)
    
    # Linie średnich
    ax.axvline(building_data['energy_consumption'].mean(), color='darkblue', linestyle='--', linewidth=1)

plt.tight_layout()
plt.show()

In [ ]:
# 1. Wybór losowego budynku
random_building = np.random.choice(df_final['building_id'].unique())

# 2. Filtrowanie danych dla wybranego budynku i roku 2017
building_data = df_final[(df_final['building_id'] == random_building) & 
                         (df_final['timestamp'].dt.year == 2017)& 
                         (df_final['timestamp'].dt.month == 3)].sort_values('timestamp')

# 3. Tworzenie wykresu scatter
plt.figure(figsize=(15, 6))
plt.plot(building_data['timestamp'], building_data['energy_consumption'], alpha=0.7)

# 4. Formatowanie wykresu
plt.title(f'Zużycie energii w czasie dla budynku {random_building} (2017)\n'
          f'Średnia temperatura: {building_data["airTemperature"].mean():.1f}°C', fontsize=14)
plt.xlabel('Czas', fontsize=12)
plt.ylabel('Zużycie energii', fontsize=12)


plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

### Analiza i preprocessing

In [ ]:
df_final.describe()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Zakładam, że df_final jest już przygotowany i zawiera wskazane kolumny

# Histogramy
df_final.hist(
    figsize=(14, 10),
    bins=30,
    edgecolor='black',
    layout=(3, 3)  # 3x3 układ wykresów
)
plt.suptitle('Histogramy kolumn', fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:
df_final = df_final[df_final['energy_consumption'] < 2000]

df_final.hist(
    figsize=(14, 10),
    bins=30,
    edgecolor='black',
    layout=(3, 3)  # 3x3 układ wykresów
)

plt.suptitle('Histogramy kolumn', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
df_final

In [ ]:
# Heatmapa korelacji
plt.figure(figsize=(10, 8))
corr_matrix = df_final.drop(columns=['timestamp', 'timezone', 'industry', 'building_id', 'site_id']).corr()  # timestamp nie jest liczbowy
sns.heatmap(
    corr_matrix,
    annot=True,
    cmap='coolwarm',
    fmt=".2f",
    square=True,
    linewidths=0.5
)
plt.title('Macierz korelacji')
plt.show()


In [ ]:
# Heatmapa korelacji po zrzuceniu pustych wartości
plt.figure(figsize=(10, 8))
corr_matrix = df_final.drop(columns=['timestamp', 'timezone', 'industry', 'building_id', 'site_id']).dropna().corr()  # timestamp nie jest liczbowy
sns.heatmap(
    corr_matrix,
    annot=True,
    cmap='coolwarm',
    fmt=".2f",
    square=True,
    linewidths=0.5
)
plt.title('Macierz korelacji')
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import gaussian_kde

# Ustawienia stylu i rozmiaru wykresów
plt.rcParams['figure.figsize'] = (16, 6)

def calculate_density_comparison(real_data, simulated_data, max_points=10000):
    """
    Funkcja obliczająca i porównująca gęstości rozkładów
    """
    # Obliczanie zakresu dla wykresów
    combined_data = np.concatenate([real_data.reshape(-1), simulated_data.reshape(-1)])
    min_val = np.min(combined_data)
    max_val = np.percentile(combined_data, 99.9)
    
    # Obliczanie gęstości
    grid = np.linspace(min_val, max_val, 500)
    kde_real = gaussian_kde(real_data)(grid)
    kde_sim = gaussian_kde(simulated_data)(grid)
    
    return grid, kde_real, kde_sim

def simulate_from_prior(sqm, temp, num_samples=1000):
    log_sqm = np.log1p(sqm)  # Używamy log1p dla bezpieczeństwa
    log_sqm_scaled = (log_sqm - np.mean(log_sqm)) / np.std(log_sqm)
    temp_scaled = (temp - np.mean(temp)) / np.std(temp)
    
    # 2. Optymalne parametry modelu
    beta0 = np.random.normal(4.0, 1, num_samples)  # Średni poziom zużycia
    beta1 = np.random.normal(0.6, 0.1, num_samples)  # Wpływ powierzchni
    beta2 = np.random.normal(-0.4, 0.1, num_samples) # Wpływ temperatury
    sigma = np.random.lognormal(-1, 1, num_samples) # Kontrola rozrzutu
    
    # 3. Generowanie z nieliniowościami
    mu = beta0[:, None] + \
         beta1[:, None] * np.tanh(log_sqm_scaled) + \
         beta2[:, None] * temp_scaled * np.exp(-temp_scaled**2)  # Lokalne ekstremum
    
    # 4. Symulacja z obcięciem ekstremów
    simulated = np.exp(np.random.normal(mu, sigma[:, None]))
    simulated = np.minimum(simulated, 2000)  # Obcięcie ekstremalnych wartości
    
    return simulated.flatten()

# Przygotowanie danych
sample_size = 1000
clean_data = df_final[["energy_consumption", "sqm", "airTemperature"]].dropna()

# Pobranie próbki zachowującej relacje między kolumnami
if len(clean_data) > sample_size:
    real_sample = clean_data.sample(n=sample_size, random_state=42)
else:
    real_sample = clean_data.copy()

# Wydzielenie kolumn
real_energy = real_sample["energy_consumption"].values
sqm = real_sample["sqm"].values
temp = real_sample["airTemperature"].values

# Generowanie symulowanych danych
simulated_data = simulate_from_prior(sqm, temp)

# Tworzenie wykresów
fig, (ax1, ax2) = plt.subplots(1, 2)

# Wykres 1: Skala liniowa
sns.histplot(real_energy, bins=50, color='blue', alpha=0.5, 
             label='Rzeczywiste', ax=ax1, stat='density', kde=True)
sns.histplot(simulated_data, bins=50, color='red', alpha=0.5, 
             label='Symulowane', ax=ax1, stat='density', kde=True)
ax1.set_title('Porównanie gęstości zużycia energii (skala liniowa)')
ax1.set_xlabel('Zużycie energii [kWh]')
ax1.set_ylabel('Gęstość prawdopodobieństwa')
ax1.legend()
ax1.grid(True)

# Ustawienie limitu osi X
combined = np.concatenate([real_energy, simulated_data])
ax1.set_xlim(0, np.percentile(combined, 99))

# Wykres 2: Skala logarytmiczna
log_real = np.log1p(real_energy)
log_sim = np.log1p(simulated_data)

sns.histplot(log_real, bins=50, color='blue', alpha=0.5, 
             label='Rzeczywiste', ax=ax2, stat='density', kde=True)
sns.histplot(log_sim, bins=50, color='red', alpha=0.5, 
             label='Symulowane', ax=ax2, stat='density', kde=True)
ax2.set_title('Porównanie rozkładów (skala log)')
ax2.set_xlabel('log(1 + zużycie energii)')
ax2.set_ylabel('Gęstość prawdopodobieństwa')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()


### Modele

In [ ]:
# Zakładamy, że mamy kolumnę np. "energy"
df_subset = df_final[['airTemperature', 'sqm', "energy_consumption", 'building_id']].dropna()

# Creating models
N = 10000
df_subset = df_subset.iloc[:N]

model_prior = CmdStanModel(stan_file='prior.stan')

temp = df_subset['airTemperature']
sqm	= df_subset['sqm']

# Setting data
data = {'N': N, 'temp': temp, 'sqm': sqm}

# Sampling z priora
prior_1 = model_prior.sample(
    data=data,
    iter_sampling=1000,
    iter_warmup=1,
    chains=1,
    fixed_param=True,
    seed=29042020,
    refresh=100,
)

# Odczyt wyników
prior_df = prior_1.draws_pd()
prior_df.head()


In [ ]:
prior_samples = prior_1.y_sim.mean(axis=0)

plt.hist(np.log1p(df_final["energy_consumption"]), bins=50, alpha=0.6, label="Dane", color='blue', density=True)
plt.hist(np.log1p(prior_samples), bins=50, alpha=0.6, label="Prior", color='orange', density=True)
plt.legend()
plt.xlabel("log(Zużycie energii)")
plt.ylabel("Gęstość")
plt.title("Porównanie na log-skali")
plt.show()

In [ ]:
prior_df.describe()

In [ ]:
y_sim_columns = [col for col in prior_df.columns if col.startswith('y_sim')]

# Spłaszcz wszystkie próbki do jednej tablicy
prior_samples = np.concatenate([prior_df[col].values for col in y_sim_columns])

# Narysuj histogram
plt.figure(figsize=(10, 6))
plt.hist(prior_samples, bins=100, color='skyblue', edgecolor='black', alpha=0.7, density=True)
plt.hist(df_final["energy_consumption"], bins=50, alpha=0.6, label="Prior", color='orange', density=True)
plt.title('Rozkład a priori symulowanego zużycia energii (y_sim < 20 000)')
plt.xlabel('Zużycie energii')
plt.ylabel('Liczba próbek')
plt.xlim(0, 500)  # Ustawienie limitu osi X

plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
az.summary(prior_1)

In [ ]:
data = {'N': N, 'temp': df_subset['airTemperature'], 'sqm': df_subset['sqm'], 'y': df_subset['energy_consumption']}
model_posterior = CmdStanModel(stan_file='posterior.stan')
post_1 = model_posterior.sample(data=data, chains=1, iter_sampling=100)

# Odczyt wyników
posterior_df = post_1.draws_pd()
posterior_df.head()

In [ ]:
posterior_samples = post_1.y_sim.flatten()

plt.hist(np.log1p(df_final["energy_consumption"]), bins=50, alpha=0.6, label="Dane", color='blue', density=True)
plt.hist(np.log1p(posterior_samples), bins=50, alpha=0.6, label="Posterior", color='orange', density=True)
plt.legend()
plt.xlabel("log(Zużycie energii + 1)")
plt.ylabel("Gęstość")
plt.title("Porównanie na log-skali")
plt.show()

In [ ]:
y_sim_columns = [col for col in posterior_df.columns if col.startswith('y_sim')]

# Spłaszcz wszystkie próbki do jednej tablicy
posterior_samples = np.concatenate([posterior_df[col].values for col in y_sim_columns])

# Narysuj histogram
plt.figure(figsize=(10, 6))
plt.hist(posterior_samples, bins=100, color='skyblue', label="Posterior", edgecolor='black', alpha=0.7, density=True)
plt.hist(df_final["energy_consumption"], bins=50, alpha=0.6, label="Real", color='orange', density=True)
plt.title('Rozkład a priori symulowanego zużycia energii (y_sim < 20 000)')
plt.xlabel('Zużycie energii')
plt.ylabel('Liczba próbek')
plt.xlim(0, 500)  # Ustawienie limitu osi X
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
az.summary(post_1)

In [ ]:
df_test = df_final[df_final['building_id'] == "Bear_assembly_Angel"]
df_test

In [ ]:
# 1. Wczytanie danych historycznych
df_subset = df_test[['airTemperature', 'sqm', 'energy_consumption']].dropna().copy()

N = 100
df_subset = df_subset.iloc[:N]

# 2. Przygotowanie danych dla modelu
data = {
    'N': len(df_subset),
    'temp': df_subset['airTemperature'].values.astype(float),
    'sqm': df_subset['sqm'].values.astype(float),
    'y': df_subset['energy_consumption'].values.astype(float)
}

# 3. Wczytanie modelu
model = CmdStanModel(stan_file='posterior.stan')

# 4. Pobranie aktualnych danych (tutaj przykładowe wartości - zastąp rzeczywistymi)
current_temp = 4.4
current_sqm = 22117

# 5. Przygotowanie danych predykcyjnych
pred_data = data.copy()
pred_data['temp'] = np.append(data['temp'][-99:], current_temp)
pred_data['sqm'] = np.append(data['sqm'][-99:], current_sqm)
pred_data['N'] = len(pred_data['temp'])

# 6. Generowanie prognozy
fit = model.sample(data=pred_data, chains=1, iter_sampling=1000)
y_sim = fit.stan_variable('y_sim')[:, -1]

# 7. Analiza wyników
print("\nWynik prognozy zużycia energii:")
print(f"- Aktualna temperatura: {current_temp}°C")
print(f"- Powierzchnia: {current_sqm} m²")
print(f"- Przewidywane zużycie energii: {np.mean(y_sim):.1f} kWh")
print(f"- 95% przedział niepewności: ({np.percentile(y_sim, 2.5):.1f}, {np.percentile(y_sim, 97.5):.1f}) kWh")

# 8. Wizualizacja rozkładu prognozy (opcjonalnie)
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 5))
plt.hist(y_sim, bins=30, color='skyblue', edgecolor='black')
plt.title('Rozkład przewidywanego zużycia energii')
plt.xlabel('Zużycie energii (kWh)')
plt.ylabel('Liczba próbek')
plt.grid(True, alpha=0.3)
plt.show()